## 如何做一个sheller

In [ ]:
maker = ReactAgentMaker()

maker.add_tool_class(ShellTools,supervision=False,verbose=True)

agent = maker.create_agent()

with open('SDV/README.md','r') as f:
    readme = f.read()

result = llm4o.complete(f"""
根据该github 中的readme 文件,帮我做一个conda环境配置方案

----------
ReadME.md

{readme}
----------
""")

print(result.text)



resultw = agent.chat(f'''
我这里有一份详细的方案,请根据方案的描述,一步一步执行方案
------------
{result.text}
''')

在Linux系统中，将自制的shell脚本添加到系统中并使其可用，通常需要以下几个步骤：

1. **创建脚本文件**：
   - 首先，你需要创建你的shell脚本文件。可以使用文本编辑器来编写脚本，例如使用`nano`或`vim`。

   ```bash
   nano myscript.sh
   ```

2. **编写脚本内容**：
   - 在文件中编写你的shell脚本代码，并确保脚本的第一行指定了解释器路径，例如：

   ```bash
   #!/bin/bash
   # 这里是你的脚本内容
   ```

3. **保存并退出编辑器**：
   - 以`nano`为例，可以使用`Ctrl + X`然后按`Y`保存更改并退出。

4. **给脚本文件添加执行权限**：
   - 使用`chmod`命令给脚本文件添加执行权限。

   ```bash
   chmod +x myscript.sh
   ```

5. **移动脚本到系统路径**：
   - 为了让脚本在任何位置都能被调用，你可以将它移动到`/usr/local/bin`或者`/usr/bin`这样的系统路径下。

   ```bash
   sudo mv myscript.sh /usr/local/bin/myscript
   ```

6. **创建符号链接**（可选）：
   - 如果你想要在多个位置都能使用这个脚本，可以创建符号链接。

   ```bash
   ln -s /usr/local/bin/myscript /usr/bin/myscript
   ```

7. **测试脚本**：
   - 在终端中运行你的脚本，确保它按预期工作。

   ```bash
   myscript
   ```

8. **添加到环境变量**（可选）：
   - 如果你想要在任何目录下都能直接调用脚本，可以将其路径添加到`PATH`环境变量中。这可以通过修改用户的shell配置文件实现，例如`.bashrc`或`.zshrc`。

   ```bash
   echo 'export PATH=$PATH:/path/to/your/script' >> ~/.bashrc
   source ~/.bashrc
   ```

9. **设置Cron作业**（如果需要定时执行）：
   - 如果你的脚本需要定期执行，可以设置cron作业。

   ```bash
   crontab -e
   ```

   然后在打开的编辑器中添加一行，如：

   ```bash
   0 * * * * /usr/local/bin/myscript
   ```

   这表示每小时执行一次脚本。

请记得，将脚本移动到系统路径或添加到环境变量时，需要管理员权限。此外，确保你对脚本的安全性和权限管理有足够的了解，以避免潜在的安全风险。

In [ ]:
#!/bin/bash
# 检查参数数量


case "$1" in
  -h|--help)
echo "work helper 一个shell脚本的便捷工具包"
echo "pip 操作"

echo """
## ssh
#    work ssh create root@192.168.8.125
#    work ssh install root@192.168.8.125
    """

    exit 0
    ;;
  -v|--version)
    echo "1.1.0"
    exit 0
    ;;
  *)
    ;;
esac


# 获取操作类型和文件名
operation=$1
operation2=$2
filename=$3
source_url=$SourceUrl #192.168.8.125:8100
#    -p8890-9000:8890-9000 \
# 根据操作类型和是否快速模式执行相应的命令
if [ "$operation" == "docker" ]; then
    if [ "$operation2" == "ps" ]; then
        docker ps --format "table {{.ID}}\t{{.Names}}\t{{.Ports}}" --all
    elif [ "$operation2" == "jupyter" ]; then
        docker run -v /Users/zhaoxuefeng/GitHub:/Users/zhaoxuefeng/GitHub \
                   -v /Users/zhaoxuefeng/Documents:/Users/zhaoxuefeng/Documents \
                   -p8888:8888 \
                   --name=$filename \
                   --rm jupyter:latest
    else
        echo "other"
    fi

elif [ "$operation" == "ssh" ]; then
    if [ "$operation2" == "create" ]; then
      bash /usr/local/bin/tools/ssher.sh create $filename
    elif [ "$operation2" == "install" ]; then
      bash /usr/local/bin/tools/ssher.sh install $filename
    else
	    echo "输入错误"
    fi
else
    echo "Unsupported operation: $operation"
    exit 1
fi





# 获取操作类型和文件名
operation=$1
operation2=$2
filename=$3
source_url=$SourceUrl #192.168.8.125:8100


# 根据操作类型和是否快速模式执行相应的命令
if [ "$operation" == "docker" ]; then
    if [ "$operation2" == "ps" ]; then
        docker ps --format "table {{.ID}}\t{{.Names}}\t{{.Ports}}" --all
    elif [ "$operation2" == "triton_vllm" ]; then
	    docker run --gpus '"device=1"' --shm-size=2g --name "$filename" \
		    -p8000:8000 -p8001:8001 -p8002:8002 \
            -v /qe/data/data/TritonConfigs:/models -w /models \
            -v /qe/data/data/Models_LLM:/llmModels \
		        nvcr.io/nvidia/tritonserver:23.12-vllm-python-py3 tritonserver --model-repository=/models --model-control-mode explicit --load-model demo

    elif [ "$operation2" == "jupyter" ]; then
        docker run -v /Users/zhaoxuefeng/GitHub:/Users/zhaoxuefeng/GitHub \
                   -v /Users/zhaoxuefeng/Documents/:/home \
                   -p8888-9000:8888-9000 \
                   -e computer_username='qe' \
                   -e computer_password='qe1234' \
                   --name='zxf_8888' \
                   --rm jupyter:latest
    elif [ "$operation2" == "project" ]; then
        docker run #TODO
    else
	    echo "输入错误"
    fi

elif [ "$operation" == "package" ]; then
    if [ "$operation2" == "update_private" ]; then
	    bash /usr/local/bin/tools/pip_source.sh private "$filename"

    elif [ "$operation2" == "install_private" ]; then
	    pip install "$filename" -i http://"$source_url"/private/simple/ --trusted-host "$source_url"

    elif [ "$operation2" == "install_cache" ]; then
	    pip install "$filename" -i http://"$source_url"/cache/simple/ --trusted-host "$source_url"
    else
	    echo "输入错误"
    fi

elif [ "$operation" == "conda" ]; then
    if [ "$operation2" == "create_stable" ]; then
	    bash /usr/local/bin/tools/pip_source.sh private "$filename"

    elif [ "$operation2" == "update_stable" ]; then
	    pip install "$filename" -i http://"$source_url"/private/simple/ --trusted-host "$source_url"

    elif [ "$operation2" == "create_test" ]; then
	    pip install "$filename" -i http://"$source_url"/cache/simple/ --trusted-host "$source_url"
    else
	    echo "输入错误"
    fi

else
    echo "输入错误"
    exit 1
fi



In [ ]:

case "$1" in
  -h|--help)
    pip cache dir  # 显示缓存的目录
    pip cache list  # 显示缓存目录中的所有文件
    pip cache purge # 清除缓存目录中的所有文件
    pip freeze > requirements.txt  # 环境导出
参数配置
    pip config set global.index-url http://{url}/simple/
    pip config set global.trusted-host {url}
安装
    pip install {package} --no-cache-dir -i http://192.168.8.125:8100/simple/ --trusted-host 192.168.8.125
    pip install -r requirements.txt
    pip download {package} -d {path}
pip2pi
    pip2pi {home_path} {package}
    pip2pi {home_path} -r requirements.txt
"""
echo "conda 操作"
echo """
    conda create -n {name} python={python}
    conda env create -f environment.yml
    conda create --name {name} --clone {origin}
    conda —c {channel} install {package}
    conda config --set show_channel_urls yes 显示conda配置中的通道URL。 设置搜索时显示通道地址
    conda config --remove-key channels
    conda config --add channels {url}
    conda config --remove channels {url}
    conda install --revision N：这个命令可以让你恢复到之前的某个版本，N是版本号，你可以用conda list --revisions查看。
    conda list --revisions：这个命令可以列出你安装或卸载的所有包的历史记录，每次改变都有一个版本号和时间戳。
    conda list -n env_name -r：这个命令可以列出某个环境下的所有包的历史记录，env_name是环境的名字，你可以用conda env list查看。
"""
echo "docker 操作"
echo """
    work docker ps
    work docker jupyter
    docker image ls
    docker image rm {imageName}


    docker version
    docker info
    docker login
    docker search {image_name}
    docker logs {container_id}
    docker start {container_id}
    docker stop {container_id}
    docker restart {container_id}
    docker kill {container_id}
    docker rm {container_id}
    docker ps --format "table {{.ID}}\t{{.Names}}\t{{.Ports}}"

    sudo usermod -aG docker {username} #将用户添加到docker用户组。
    docker pull {name_version} #从Docker Hub拉取一个镜像。
    docker push {name_version} 将一个镜像推送到Docker Hub。
    docker exec -it {container_id} /bin/bash
    docker commit -m {message} -a "zhaoxuefeng" {container_id} {name_version} 提交一个容器的更改。
    docker build -t {name_version} .

"""
echo "jupyter"

echo """
    jupyter
    jupyter lab password
    jupyter server password
    jupyter lab --generate-config
    jupyter lab --notebook-dir={notebook_dir} --port {port} --root-allow

    pycharm风格设置
    #pycharm 的 File -> Settings -> Plugins,搜索Material Theme UI 并安装,安装之后进行restart
    #02字体:File -> Settings -> Editor -> Font, Font: Source Code Pro,Size: 16, line-spacing: 1.0, apply,保存 .
    #03字体颜色:File -> Settings -> Editor -> Color Scheme Font -> General, scheme选择Darcula, apply,保存 .( 这里我是设置的黑色,如果喜欢白色的可以选择github更佳美观 )
    #04背景图片:设置的路径为:File | Settings | Appearance & Behavior | Appearance选择Background Image
    """

echo "user"
echo """
    #sudo adduser {username}
    #sudo passwd {username}
    #passwd {username}
    """

echo "linux"
echo """
    #ps -ef | grep {progress}
    #lsof -i:{port}
    #kill -9 {pid}
    #tar -zcvf {path}.tar.gz {path}
    #tar -zxvf {tgz_path}
    #zip {files}
    #unzip -O CP936 {files}
    #lspci | grep NVIDIA
    #uname -a 查看Ubuntu版本信息

    #初始化Ubuntu环境
    #sudo apt update
    #sudo apt install build-essential
    #sudo apt install python3.8
    #sudo apt install pciutils
    #
    #ln -s {origin_path} {to_path}
    #wget -c {url}
    #ssh -N -f -L localhost:$1:localhost:$1 root@$2
    #alias {name}='{order}'
    #rsync -avh --progress --partial /path/to/source user@remote:/path/to/destination
    #

#cache缓存替换
#mv ~/.cache /path/to/new/cache
#ln -s /path/to/new/cache ~/.cache
#
#sudo sshfs -o cache=yes,allow_other {target} {a}
#du -sh {file_path}
#sudo mount {a} {b}
#chmod -R 777 {file_path}
#mkdir -p {file_path}
#history {n}
#history -c
## 默认注释了源码镜像以提高 apt update 速度，如有需要可自行取消注释
#deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy main restricted universe multiverse
## deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy main restricted universe multiverse
#deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-updates main restricted universe multiverse
## deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-updates main restricted universe multiverse
#deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-backports main restricted universe multiverse
## deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-backports main restricted universe multiverse
#deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-security main restricted universe multiverse
## deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-security main restricted universe multiverse
## 预发布软件源，不建议启用
## deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-proposed main restricted universe multiverse
## deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-proposed main restricted universe multiverse
#>> /etc/apt/sources.list

"""

echo """
## ssh
#    work ssh create root@192.168.8.125
#    work ssh install root@192.168.8.125
    """
    echo """
#tmux ls
#tmux new -s {session_name}
#tmux detach
#tmux attach -t {session_name}
#tmux kill-session -t {session_name}
#tmux switch -t {session_name}
#tmux rename-session -t {session_name} {new_name}
#tmux send-keys -t {session}:{window} "{order}" C-m
#
#        #%：划分左右两个窗格。
#        <arrow key>：光标切换到其他窗格。
#        ;：光标切换到上一个窗格。
#        o：光标切换到下一个窗格。
#        {：当前窗格与上一个窗格交换位置。
#        }：当前窗格与下一个窗格交换位置。
#        Ctrl+o：所有窗格向前移动一个位置，第一个窗格变成最后一个窗格。
#        Alt+o：所有窗格向后移动一个位置，最后一个窗格变成第一个窗格。
#        x：关闭当前窗格。
#        !：将当前窗格拆分为一个独立窗口。
#        z：当前窗格全屏显示，再使用一次会变回原来大小。
#        Ctrl+<arrow key>：按箭头方向调整窗格大小。
#        q：显示窗格编号。

    """
echo """
#    cuda
关闭系统驱动
sudo sh -c \"echo 'blacklist nouveau' > /etc/modprobe.d/blacklist-nouveau.conf\"
sudo sh -c \"echo 'options nouveau modeset=0' >> /etc/modprobe.d/blacklist-nouveau.conf\"
sudo update-initramfs -u
sudo reboot

#    lsmod | grep nouveau 检查是否已经关闭驱动

#    验证安装
#     nvcc -V
      nvidia-smi
      python -c 'import torch;print(torch.cuda.is_available())'
      python -c 'import torch;a = torch.ones(1,2,3,4);print(a.to(0))'

#watch -n 2 -d nvidia-smi 查看gpu状态# 每隔2秒刷新一次，每次只在固定位置刷新
#CUDA_VISIBLE_DEVICES=0 {text}
#!find /qe/data/data -name package
#%cat /sys/class_level/thermal/thermal_zone0/temp
#
#

配置
#
#nvidia-smi No devices were found
#sudo add-apt-repository ppa:graphics-drivers/ppa #不知是否必要
#sudo apt-get install ubuntu-drivers
#ubuntu-drivers devices
#执行结果
#sudo apt install -y nvidia-driver-525
#sudo reboot
#
#\"echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64'>> ~/.bashrc\"
#\"echo 'export PATH=$PATH:/usr/local/cuda/bin'>> ~/.bashrc\"
#\"echo 'export CUDA_HOME=$CUDA_HOME:/usr/local/cuda'>> ~/.bashrc\"
#source ~/.bashrc
#sudo ldconfig


#   def cudnn(self):
#
#        • sudo dpkg -i cudnn-local-repo-ubuntu2004-8.9.6.50_1.0-1_amd64.deb
#        • sudo cp /var/cudnn-local-repo-ubuntu2004-8.9.6.50_1.0-1_amd64/cudnn-local-4B348671-keyring.gpg /usr/share/keyrings/
#        • sudo apt update
#
#        • sudo apt install libcudnn8
#        • sudo apt install libcudnn8-dev
#        • sudo apt install libcudnn8-samples
#
#        from torch.backends import cudnn
#        assert isinstance(cudnn.version(),int)
#        assert cudnn.is_available() == True
#        a=torch.tensor ( 1.)
#        assert cudnn.is_acceptable (a.cuda ()) ==True
#
#
#    def tensorrt(self):
#
#        sudo dpkg -i tensorrt.deb
#        sudo cp /var/cudnn-local-repo-ubuntu2004-8.9.6.50_1.0-1_amd64/cudnn-local-4B348671-keyring.gpg /usr/share/keyrings/
#        sudo apt update
#        sudo apt install tensorrt

"""


    exit 0
    ;;
  -v|--version)
    echo "1.1.0"
    exit 0
    ;;
  *)
    ;;
esac


# 获取操作类型和文件名
operation=$1
operation2=$2
filename=$3
source_url=$SourceUrl #192.168.8.125:8100
#    -p8890-9000:8890-9000 \
# 根据操作类型和是否快速模式执行相应的命令
if [ "$operation" == "docker" ]; then
    if [ "$operation2" == "ps" ]; then
        docker ps --format "table {{.ID}}\t{{.Names}}\t{{.Ports}}" --all
    elif [ "$operation2" == "jupyter" ]; then
        docker run -v /Users/zhaoxuefeng/GitHub:/Users/zhaoxuefeng/GitHub \
                   -v /Users/zhaoxuefeng/Documents:/Users/zhaoxuefeng/Documents \
                   -p8888:8888 \
                   --name=$filename \
                   --rm jupyter:latest
    else
        echo "other"
    fi

elif [ "$operation" == "ssh" ]; then
    if [ "$operation2" == "create" ]; then
      bash /usr/local/bin/tools/ssher.sh create $filename
    elif [ "$operation2" == "install" ]; then
      bash /usr/local/bin/tools/ssher.sh install $filename
    else
	    echo "输入错误"
    fi
else
    echo "Unsupported operation: $operation"
    exit 1
fi





# 获取操作类型和文件名
operation=$1
operation2=$2
filename=$3
source_url=$SourceUrl #192.168.8.125:8100


# 根据操作类型和是否快速模式执行相应的命令
if [ "$operation" == "docker" ]; then
    if [ "$operation2" == "ps" ]; then
        docker ps --format "table {{.ID}}\t{{.Names}}\t{{.Ports}}" --all
    elif [ "$operation2" == "triton_vllm" ]; then
	    docker run --gpus '"device=1"' --shm-size=2g --name "$filename" \
		    -p8000:8000 -p8001:8001 -p8002:8002 \
            -v /qe/data/data/TritonConfigs:/models -w /models \
            -v /qe/data/data/Models_LLM:/llmModels \
		        nvcr.io/nvidia/tritonserver:23.12-vllm-python-py3 tritonserver --model-repository=/models --model-control-mode explicit --load-model demo

    elif [ "$operation2" == "jupyter" ]; then
        docker run -v /Users/zhaoxuefeng/GitHub:/Users/zhaoxuefeng/GitHub \
                   -v /Users/zhaoxuefeng/Documents/:/home \
                   -p8888-9000:8888-9000 \
                   -e computer_username='qe' \
                   -e computer_password='qe1234' \
                   --name='zxf_8888' \
                   --rm jupyter:latest
    elif [ "$operation2" == "project" ]; then
        docker run #TODO
    else
	    echo "输入错误"
    fi

elif [ "$operation" == "package" ]; then
    if [ "$operation2" == "update_private" ]; then
	    bash /usr/local/bin/tools/pip_source.sh private "$filename"

    elif [ "$operation2" == "install_private" ]; then
	    pip install "$filename" -i http://"$source_url"/private/simple/ --trusted-host "$source_url"

    elif [ "$operation2" == "install_cache" ]; then
	    pip install "$filename" -i http://"$source_url"/cache/simple/ --trusted-host "$source_url"
    else
	    echo "输入错误"
    fi

elif [ "$operation" == "conda" ]; then
    if [ "$operation2" == "create_stable" ]; then
	    bash /usr/local/bin/tools/pip_source.sh private "$filename"

    elif [ "$operation2" == "update_stable" ]; then
	    pip install "$filename" -i http://"$source_url"/private/simple/ --trusted-host "$source_url"

    elif [ "$operation2" == "create_test" ]; then
	    pip install "$filename" -i http://"$source_url"/cache/simple/ --trusted-host "$source_url"
    else
	    echo "输入错误"
    fi

else
    echo "输入错误"
    exit 1
fi



